<a href="https://colab.research.google.com/github/ISaySalmonYouSayYes/Damage-Level-Classification-from-Images-Using-Deep-Learning-with-Data-Augmentation/blob/main/damageDetectingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Before you Start**

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    drive.mount('/content/drive')
    print("Note: using Google CoLab")
    print("Number of CPU cores (os):", os.cpu_count())
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab
Number of CPU cores (os): 12


In [ ]:
!tar -xvzf /content/drive/MyDrive/ASONAM17_Damage_Image_Dataset.tar.gz -C /content/ #90sec

# **1. Data preparation**
- Modify transform parameter to deploy augmentation

In [72]:
import os
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image


class UnifiedDataset(Dataset):
    def __init__(self, data_source, root_dir=None, transform=None, is_file_based=False):
        """
        A unified dataset class to handle both directory-based (ImageFolder-like)
        and file-list-based (CustomDataset-like) datasets.

        Args:
        - data_source (str): Either a directory path (for ImageFolder) or a file path (for .train, .dev, .test).
        - root_dir (str, optional): Root directory for file-based datasets.
        - transform (callable, optional): Transformations to apply to the images.
        - is_file_based (bool): Whether the data_source is a file (True) or a directory (False).
        """
        self.transform = transform
        self.is_file_based = is_file_based

        if is_file_based:
            # For file-based datasets (.train, .dev, .test)
            self.root_dir = root_dir
            with open(data_source, "r") as f:
                self.data = [line.strip().split() for line in f]
        else:
            # For directory-based datasets (ImageFolder-like)
            self.dataset = datasets.ImageFolder(root=data_source, transform=transform)

    def __len__(self):
        if self.is_file_based:
            return len(self.data)
        return len(self.dataset)

    def __getitem__(self, idx):
        if self.is_file_based:
            # File-based loading
            img_path, label = self.data[idx]
            img_path = os.path.join(self.root_dir, img_path)
            image = Image.open(img_path).convert("RGB")  # Ensure it's RGB
            label = int(label)  # Convert label to integer
            if self.transform:
                image = self.transform(image)
            return image, label
        else:
            # Directory-based loading (delegates to ImageFolder)
            return self.dataset[idx]


# Define transformations
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(
        brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
    ),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0))], p=0.5),
    transforms.ToTensor(),  # Convert to Tensor here
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3)),  # RandomErasing after ToTensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize after ToTensor
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Define dataset paths
train_file = "/content/ASONAM17_Damage_Image_Dataset/ruby.train"
dev_file = "/content/ASONAM17_Damage_Image_Dataset/ruby.dev"
test_file = "/content/ASONAM17_Damage_Image_Dataset/ruby.test"
root_dir = "/content/ASONAM17_Damage_Image_Dataset"

# Modify transform parameter to deploy augmentation
train_dataset = UnifiedDataset(data_source=train_file, root_dir=root_dir, transform=train_transforms, is_file_based=True)
dev_dataset = UnifiedDataset(data_source=dev_file, root_dir=root_dir, transform=val_transforms, is_file_based=True)
test_dataset = UnifiedDataset(data_source=test_file, root_dir=root_dir, transform=val_transforms, is_file_based=True)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=12)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, num_workers=12)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=12)

# Check data loader
for images, labels in train_loader:
    print("Batch shape:", images.shape)  # Example: torch.Size([32, 3, 224, 224])
    print("Labels:", labels)
    break

Batch shape: torch.Size([32, 3, 224, 224])
Labels: tensor([2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0])


# **2. Model**

In [73]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load ResNet model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)  # 3 classes (0 to 2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [74]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [75]:
from tqdm import tqdm
import copy

def train_model(model, train_loader, dev_loader, criterion, optimizer, epochs=10, patience=5):
    best_val_loss = float("inf")  # Initialize best validation loss
    best_model_state = None       # Store the best model state
    patience_counter = 0          # Tracks how many epochs since the last improvement

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        train_loader_tqdm = tqdm(train_loader, desc="Training", leave=True)
        for images, labels in train_loader_tqdm:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            train_loader_tqdm.set_postfix(loss=loss.item(), accuracy=correct / total)

        train_accuracy = correct / total
        print(f"Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_accuracy:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in dev_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_accuracy = correct / total
        val_loss /= len(dev_loader)
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

        # Save the best model state in memory (based on validation loss)
        if val_loss < best_val_loss:
            print(f"Updating best model... (Val Loss: {val_loss:.4f})")
            best_val_loss = val_loss
            best_model_state = copy.deepcopy(model.state_dict())  # Save the state dict
            patience_counter = 0  # Reset the patience counter
        else:
            patience_counter += 1  # Increment the patience counter
            print(f"No improvement in validation loss. Patience: {patience_counter}/{patience}")

        # Check if early stopping is triggered
        if patience_counter >= patience:
            print("Early stopping triggered. Restoring best model...")
            break

    # Restore the best model state after training
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print("Model restored to the best state (based on validation loss).")

    print("Training complete.")
    return model  # Return the best version of the model

## **2.1 Train: last layer**

In [70]:
train_model(model, train_loader, dev_loader, criterion, optimizer, epochs=100, patience=15)

Epoch 1/100


Training: 100%|██████████| 16/16 [00:03<00:00,  4.42it/s, accuracy=0.488, loss=1.32]

Train Loss: 1.2493, Train Acc: 0.4880


Val Loss: 6.1027, Val Acc: 0.4251
Updating best model... (Val Loss: 6.1027)
Epoch 2/100


Training: 100%|██████████| 16/16 [00:03<00:00,  4.21it/s, accuracy=0.576, loss=0.799]

...

Val Loss: 0.8620, Val Acc: 0.6587
No improvement in validation loss. Patience: 14/15
Epoch 32/100


Training: 100%|██████████| 16/16 [00:03<00:00,  4.43it/s, accuracy=0.674, loss=0.688]

Train Loss: 0.7324, Train Acc: 0.6740


Val Loss: 1.0206, Val Acc: 0.5808
No improvement in validation loss. Patience: 15/15
Early stopping triggered. Restoring best model...
Model restored to the best state (based on validation loss).
Training complete.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## **2.2 Train: enable fine-tuning**

In [76]:
for param in model.parameters():
    param.requires_grad = True  # Unfreeze all layers

optimizer = optim.Adam(model.parameters(), lr=1e-5)
train_model(model, train_loader, dev_loader, criterion, optimizer, epochs=100, patience=15)


# fine tuning

Epoch 1/100


Training: 100%|██████████| 16/16 [00:03<00:00,  4.44it/s, accuracy=0.354, loss=1.04]

Train Loss: 1.1133, Train Acc: 0.3540


Val Loss: 1.0490, Val Acc: 0.5210
Updating best model... (Val Loss: 1.0490)
Epoch 2/100


Training: 100%|██████████| 16/16 [00:03<00:00,  4.45it/s, accuracy=0.526, loss=0.973]

...

Val Loss: 0.6783, Val Acc: 0.7485
No improvement in validation loss. Patience: 14/15
Epoch 30/100


Training: 100%|██████████| 16/16 [00:03<00:00,  4.39it/s, accuracy=0.842, loss=0.619]

Train Loss: 0.3792, Train Acc: 0.8420


Val Loss: 0.6685, Val Acc: 0.7485
No improvement in validation loss. Patience: 15/15
Early stopping triggered. Restoring best model...
Model restored to the best state (based on validation loss).
Training complete.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# **3. Testing**

In [77]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    print(f"Test Accuracy: {correct / total:.4f}")

evaluate_model(model, test_loader)

Test Accuracy: 0.8012


# **4. Save and reuse the model**

In [29]:
torch.save(model.state_dict(), "damage_detection_model.pth")

In [30]:
model.load_state_dict(torch.load("damage_detection_model.pth"))
model.eval()

<ipython-input-30-2e9032f9292a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("damage_detection_model.pth"))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## **4.1 Evaluate single image**

In [31]:
from PIL import Image
import torch

def evaluate_single_image(model, image_path, label, transform, device):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")  # Ensure it's in RGB format
    image = transform(image)  # Apply the validation transforms
    image = image.unsqueeze(0)  # Add batch dimension: [1, C, H, W]

    # Move to the correct device
    image = image.to(device)
    label = torch.tensor([label]).to(device)  # Convert label to tensor

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        outputs = model(image)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get predicted class

    # Print result
    print(f"Ground Truth Label: {label.item()}")
    print(f"Predicted Label: {predicted.item()}")
    print(f"Correct Prediction: {predicted.item() == label.item()}")

In [32]:
image_path = "/content/concrete.png"
label = 0
evaluate_single_image(model, image_path, label, val_transforms, device)

Ground Truth Label: 0
Predicted Label: 0
Correct Prediction: True
